# Imports

In [1]:
#mlops environment
import pandas as pd 
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer


In [4]:
df_jan = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
df_feb = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

In [5]:
# How many columns are there?
len(list(df_jan.columns))

19

In [6]:
print(df_jan.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


In [7]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [8]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.seconds
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.seconds


In [9]:
df_jan['duration'] = df_jan['duration']/60
df_feb['duration'] = df_feb['duration']/60

print(df_jan.duration)

0          17.816667
1           8.400000
2           8.966667
3          10.033333
4          37.533333
             ...    
2463926     5.966667
2463927    10.650000
2463928    11.000000
2463929    12.050000
2463930    27.000000
Name: duration, Length: 2463931, dtype: float64


In [10]:
df_jan.duration.value_counts()

duration
7.000000       3760
9.000000       3583
8.000000       3563
6.000000       3536
10.000000      3501
               ... 
1063.216667       1
1391.666667       1
122.050000        1
1431.950000       1
120.550000        1
Name: count, Length: 8155, dtype: int64

In [11]:
#What's the standard deviation of the trips duration in January?
np.std(df_jan.duration)

56.18569549098241

In [12]:
#What fraction of the records left after you dropped the outliers?
no_outlier = df_jan[(df_jan['duration']>=1) & (df_jan['duration']<=60)]
(no_outlier.shape[0]/df_jan.shape[0])*100

98.27551989077617

In [13]:
df_train = df_jan[['PULocationID', 'DOLocationID']]

In [14]:
df_train.dropna(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_7492\370300940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.dropna(inplace=True)


In [15]:
df_train

,PULocationID,DOLocationID
0,142,236
1,236,42
2,166,166
3,114,68
4,68,163
...,...,...
2463926,90,170
2463927,107,75
2463928,113,246
2463929,148,164


In [16]:
x_train=df_train.to_dict('records')


In [19]:
len(x_train)

2463931

In [20]:
dv = DictVectorizer()
dv.fit(x_train)
xt=pd.DataFrame(dv.fit_transform(x_train).todense(), columns=dv.feature_names_)

In [23]:
x=dv.fit_transform(x_train)
print(x)

  (0, 0)	236.0
  (0, 1)	142.0
  (1, 0)	42.0
  (1, 1)	236.0
  (2, 0)	166.0
  (2, 1)	166.0
  (3, 0)	68.0
  (3, 1)	114.0
  (4, 0)	163.0
  (4, 1)	68.0
  (5, 0)	161.0
  (5, 1)	138.0
  (6, 0)	87.0
  (6, 1)	233.0
  (7, 0)	152.0
  (7, 1)	238.0
  (8, 0)	236.0
  (8, 1)	166.0
  (9, 0)	141.0
  (9, 1)	236.0
  (10, 0)	229.0
  (10, 1)	141.0
  (11, 0)	90.0
  (11, 1)	114.0
  (12, 0)	113.0
  :	:
  (2463918, 1)	237.0
  (2463919, 0)	236.0
  (2463919, 1)	170.0
  (2463920, 0)	229.0
  (2463920, 1)	189.0
  (2463921, 0)	261.0
  (2463921, 1)	158.0
  (2463922, 0)	79.0
  (2463922, 1)	233.0
  (2463923, 0)	116.0
  (2463923, 1)	246.0
  (2463924, 0)	75.0
  (2463924, 1)	18.0
  (2463925, 0)	90.0
  (2463925, 1)	224.0
  (2463926, 0)	170.0
  (2463926, 1)	90.0
  (2463927, 0)	75.0
  (2463927, 1)	107.0
  (2463928, 0)	246.0
  (2463928, 1)	113.0
  (2463929, 0)	164.0
  (2463929, 1)	148.0
  (2463930, 0)	181.0
  (2463930, 1)	186.0


In [51]:
# transformer = make_column_transformer(
#     (OneHotEncoder(), ['PULocationID', 'DOLocationID']),
#     remainder='passthrough')

# transformed = transformer.fit_transform(df_train)
# transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
# print(transformed_df.head())

df_train
train =pd.get_dummies(df_train,columns=['PULocationID', 'DOLocationID'])

In [65]:
print(train.shape)
Y = df_jan.duration

(2463931, 518)


In [63]:
y_train_pred = model.predict(df_train)
rmse = mean_squared_error(Y, y_train_pred, squared=False)

In [64]:
#What's the RMSE on train?
print(rmse)

56.16520356247583
